In [5]:
import pandas as pd
import unittest
import numpy as np
np.random.seed(1)

In [6]:
%run Methods.ipynb

test_prepData (__main__.TestDataPrep) ... ok
test_withNA (__main__.TestDataPrep) ... ok
test_getItemsThatHaveElement (__main__.TestGetItemWithElements) ... ok
test_yieldPrepData (__main__.TestYieldDataPrep) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.090s

OK


In [7]:
testData = {'Area': (['Indonesia'] * 3 + ['China'] * 3 + ['UK'] * 3) * 3,
            'Item': ['Apples'] * 9 + ['Grapes'] * 9 + ['Beehives'] * 9,
            'Unit': ['hg/ha', 'tonnes', 'ha'] * 6 + ["Unit"] * 9,
            'Element': ['Yield', 'Production', 'Area'] * 6 + ['Yield', 'Stocks', 'Count'] * 3,
           '2019': np.random.randint(100, 100000, size = 27), '2020': np.random.randint(100, 100000, size = 27)}

testDf = pd.DataFrame(testData)
testDf.head(27)

,Area,Item,Unit,Element,2019,2020
0,Indonesia,Apples,hg/ha,Yield,98639,19533
1,Indonesia,Apples,tonnes,Production,77808,17237
2,Indonesia,Apples,ha,Area,5292,26203
3,China,Apples,hg/ha,Yield,98147,44610
4,China,Apples,tonnes,Production,50157,51140
5,China,Apples,ha,Area,73449,33978
6,UK,Apples,hg/ha,Yield,21540,99435
7,UK,Apples,tonnes,Production,98548,37283
8,UK,Apples,ha,Area,20709,3706
9,Indonesia,Grapes,hg/ha,Yield,49200,68197


In [8]:
def itemToProductionElementDictionary(agDF):
    """
        Create a dictionary mapping each item
        to its production element. 
        Args:
            agDF: Agriculture dataframe from FAO
        Returns: a dictionary similar to {'Apples': 'Production', 'Beehives': 'Stocks'}
    """
    
    productionElements = ["Production", "Stocks", "Laying", "Prod Popultn"]
    
    dfProdElements = agDF[agDF['Element'].isin(productionElements)]
    itemsList = dfProdElements['Item']
    elementsList = dfProdElements['Element']
    
    itemProdElementDict = {}
    
    for itemElementTuple in zip(itemsList, elementsList):
        itemProdElementDict[itemElementTuple[0]] = itemElementTuple[1]
    
    return itemProdElementDict

In [17]:
def helperGetYearVals(df, year):
    """
        Get the YEAR column in the dataframe.
        Set NAN to zero. 
        Arguments:
            df: dataframe
            year: integer
        Returns: a numpy array
    """
    yearProd = df["Y" + str(year)]
    yearProd[np.isnan(yearProd)] = 1
    return yearProd

def productionChangeDF(agDF, y1, y2):
    """
        Get a dataframe of ["Item", "Element", "Unit", "Production Change"]
        where Production Change is the percentage change of production
        of an item between years y1 and y2.
        Args: 
            agDF: Agriculture dataframe from FAO
            y1: int, the start year
            y2: int, the end year
        Returns: a dataframe 
    """
    
    checkRange(y1, y2)
    
    itemToProdElement = itemToProductionElementDictionary(agDF)
    
    itemCol = agDF['Item']
    elementCol = agDF['Element']
    selectRows = []
    
    for item, ele in zip(itemCol, elementCol):
        if (ele == itemToProdElement[item]):
            selectRows.append(True)
        else:
            selectRows.append(False)
    df = dropRegionRows(agDF)
    df = df.drop(df[df['Area'] == "China"].index)
    df = agDF.loc[selectRows]
    yearCol = createYearList(y1, y2)
    df = df[['Area', 'Item', 'Element', "Unit"] + ["Y" + str(y1), "Y" + str(y2)]]
    y1Prod = helperGetYearVals(df, y1)
    y2Prod = helperGetYearVals(df, y2)
    df['Percent Change'] = [(y2Prod[i] - y1Prod[i]) / y2Prod[i] for i in range(len(y1Prod))]
    
        
    
   
    


In [15]:
class TestItemToProdElement(unittest.TestCase):
    def createTestData(self):
        """
            Create dataframe for testing
        """
        testData = {'Area Code': ([1] * 3 +[2] * 3 + [3] * 3) * 3,
                    'Area': (['Indonesia'] * 3 + ['China'] * 3 + ['UK'] * 3) * 3,
            'Item': ['Apples'] * 9 + ['Grapes'] * 9 + ['Beehives'] * 9,
            'Element': ['Yield', 'Production', 'Area'] * 6 + ['Yield', 'Stocks', 'Count'] * 3,
            'Unit': ['hg/ha', 'tonnes', 'ha'] * 6 + ["unit"] * 9,        
           'Y2019': np.random.randint(100, 100000, size = 27), 'Y2020': np.random.randint(100, 100000, size = 27)}
        testDf = pd.DataFrame(testData)
        return testDf
    def test_itemToProductionElementDictionary(self):
        testDf = self.createTestData()
        itemToProdElementDict = itemToProductionElementDictionary(testDf)
        expected = {'Apples': 'Production', 'Grapes': 'Production', 'Beehives': 'Stocks'}
        self.assertEqual(expected, itemToProdElementDict)
    
    def test_productionChange(self):
        testDf = self.createTestData()
        expectedData = {'Item': ['Apples', 'Grapes', 'Beehives'], 'Unit': ['tonnes', 'tonnes', 'unit'], 
                       'Element': ["Production", "Production", "Stocks"], 
                       'Percent Change': ["0.47", "0.15", "0.17"]}
        expectedDF = pd.DataFrame(expectedData)
        self.assertTrue(expectedDF.equals(productionChangeDF(testDf, 2019, 2020)))

In [16]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_prepData (__main__.TestDataPrep) ... ok
test_withNA (__main__.TestDataPrep) ... ok
test_getItemsThatHaveElement (__main__.TestGetItemWithElements) ... ok
test_itemToProductionElementDictionary (__main__.TestItemToProdElement) ... ok
test_productionChange (__main__.TestItemToProdElement) ... FAIL
test_yieldPrepData (__main__.TestYieldDataPrep) ... 

       Item   Year  Amount
0    Apples  Y2019  196783
1    Apples  Y2020  110344
2  Beehives  Y2019  132894
3  Beehives  Y2020   93930
4    Grapes  Y2019  207422
5    Grapes  Y2020  164596


ok

FAIL: test_productionChange (__main__.TestItemToProdElement)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\14159\AppData\Local\Temp\ipykernel_80580\1875952709.py", line 26, in test_productionChange
    self.assertTrue(expectedDF.equals(productionChangeDF(testDf, 2019, 2020)))
AssertionError: False is not true

----------------------------------------------------------------------
Ran 6 tests in 0.112s

FAILED (failures=1)
